In [ ]:
import math
import pdb
import platform

import ai2thor
import copy
import time
import random
import ai2thor.controller
import datetime
import cv2
import os
import matplotlib

import matplotlib.pyplot as plt
import os
import numpy as np
# from utils.mid_level_constants import  scene_start_cheating_init_pose
import sys
from IPython.display import Image, display
import PIL.Image
sys.path.append('../')

from scripts.jupyter_helper import get_reachable_positions
from scripts.stretch_jupyter_helper import two_dict_equal, ARM_MOVE_CONSTANT, get_current_arm_state, only_reset_scene, transport_wrapper, ADITIONAL_ARM_ARGS, execute_command, WRIST_ROTATION, get_current_wrist_state
from utils.stretch_utils.stretch_constants import STRETCH_ENV_ARGS
matplotlib.use('TkAgg')
screen_size=224

STRETCH_ENV_ARGS['width'] = screen_size
STRETCH_ENV_ARGS['height'] = screen_size
STRETCH_ENV_ARGS['agentMode']='stretch'
STRETCH_ENV_ARGS['commit_id']='7184aa455bc21cc38406487a3d8e2d65ceba2571'
STRETCH_ENV_ARGS['renderDepthImage'] = True
STRETCH_ENV_ARGS['renderInstanceSegmentation'] = True


In [ ]:

action_options = ['m', 'r', 'l', 'b', 'hu', 'hd', 'ao', 'ai', 'go', 'gc', 'wp', 'wn']

kitchens = [f"FloorPlan{i}" for i in range(1, 31)]
living_rooms = [f"FloorPlan{200 + i}" for i in range(1, 31)]
bedrooms = [f"FloorPlan{300 + i}" for i in range(1, 31)]
bathrooms = [f"FloorPlan{400 + i}" for i in range(1, 31)]

all_scenes = kitchens + living_rooms + bedrooms + bathrooms
NUM_TESTS = 100
EPS_LEN = 500

def setup_thirdparty_camera(controller, camera_position):
    # controller.step('Pass')
    if len(controller.last_event.third_party_camera_frames) > 1:
        controller.step('UpdateThirdPartyCamera',
            thirdPartyCameraId=1, # id is available in the metadata response
            rotation=camera_position['rotation'],
            position=camera_position['position']
            )
    else:
        controller.step('AddThirdPartyCamera', 
            rotation=camera_position['rotation'], 
            position=camera_position['position'],
            fieldOfView=100)

def manual_task(controller, scene_name, logger_number =0, final=False, save_frames = False, init_sequence=[], verbose = False):
    only_reset_scene(controller, scene_name)
    all_actions = []
    all_action_details = []
    actions_ran_so_far = 0
    while(True):
        visualize(controller, save_frames)
        if len(init_sequence) > 0:
            action = init_sequence[0]
            init_sequence = init_sequence[1:]
        else:
            action = input('action?')
        if action == 'q':
            break
        all_actions.append(action)

        detail = execute_command(controller, action, ADITIONAL_ARM_ARGS)
        if verbose:
            print(detail, controller.last_event.metadata['lastActionSuccess'])
        all_action_details.append(detail)
        actions_ran_so_far += 1

        # controller.step(action='MoveArm', coordinateSpace="wrist", position=dict(x=0, y=0.2,z=0.2));visualize(controller, save_frames)
        # controller.step(action='RotateWristRelative', yaw=90);visualize(controller, save_frames)

    print(scene_name)
    print(all_actions)
    print(all_action_details)

def visualize(controller, save_frame=False):
    image = controller.last_event.frame
    arm_view = controller.last_event.third_party_camera_frames[0]
#     third_view = controller.last_event.third_party_camera_frames[1]

    combined=np.concatenate([image, arm_view], 1)
    im = PIL.Image.fromarray(combined)
    display(im)
    
    if STRETCH_ENV_ARGS['renderInstanceSegmentation']:
        print(set([k.split('|')[0] for k in controller.last_event.instance_masks.keys()]))
        print(set([k.split('|')[0] for k in controller.last_event.third_party_instance_masks[0].keys()]))
    
    if save_frame:
        saved_image_folder = '/Users/kianae/Desktop/saved_stretch_images_jupyter'
        os.makedirs(saved_image_folder, exist_ok=True)
        imagename = datetime.datetime.now().strftime("%m_%d_%Y_%H_%M_%S.%f")
        plt.imsave(os.path.join(saved_image_folder, imagename+'.png'), combined)
    



In [ ]:
controller = ai2thor.controller.Controller(**STRETCH_ENV_ARGS)

In [ ]:
# manual_task(controller, 'FloorPlan15', logger_number =0, init_sequence=['m', 'm', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'l', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', '/', 'zo', 'zo', 'zp', 'zp', 'ao', 'ao', 'ao', 'r', 'ao', 'p', 'hd', 'ao', 'ao', 'ao', 'ao', 'hd', 'hd', 'hd', '', 'hd', 'hd', 'ao', 'ao', 'wp', 'wn', 'wn', 'wn', 'wn', '/', 'p', 'u', 'u', 'hu', 'hu', 'r', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'd', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'r', 'r', 'l', 'ao', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'dh', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'ao', 'ao', 'ao', 'ao', 'l', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'ao', 'ao', '/', 'p', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'd', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'm', '', 'mm', 'm', 'm', 'm', 'm', 'm', 'm', 'm'], save_frames=True, final=True)
manual_task(controller, 'FloorPlan15', logger_number =0, init_sequence=['m', 'm', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'l', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', '/', 'zo', 'zo', 'zp', 'zp', 'ao', 'ao', 'ao', 'r', 'ao', 'p', 'hd', 'ao', 'ao', 'ao', 'ao', 'hd', 'hd', 'hd', '', 'hd', 'hd', 'ao', 'ao', 'wp', 'wn', 'wn', 'wn', 'wn', '/', 'p', 'u', 'u', 'hu', 'hu', 'r', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'd', 'ai', 'ai', 'ai', 'ai', 'ai', 'ai', 'r', 'r', 'l', 'ao', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'dh', 'hd', 'hd', 'hd', 'hd', 'hd', 'dh', 'ao', 'ao', 'ao', 'ao', 'l', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'hd', 'ao', 'ao', '/', 'p', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'd', 'r', 'r', 'r', 'r', 'm', 'm', 'm', 'm', 'm', 'm', '', 'mm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'r', 'r', 'r', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'u', 'u', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'r', 'ao', 'p', 'u', 'u', 'hu', 'hu', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu'], save_frames=True, final=False)

In [ ]:
manual_task(controller, 'FloorPlan15', logger_number =0, init_sequence=['m', 'm', 'r', 'r', 'm'])

In [ ]:

# controller.last_event.third_party_instance_masks
